# Exploring Eco topics with Python | TEST NOTEBOOK - Wildlife Trade Exploration with Python
---

## A.

In [ ]:
import pandas as pd 
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
raw_data = pd.read_csv("WildlifeTradeExp_DATA.csv")

In [ ]:
df_prep = raw_data.copy()

## B.

In [ ]:
df_prep.shape

In [ ]:
df_prep.dtypes

## C.

In [ ]:
df_prep = df_prep[df_prep["Purpose"].isin(["H", "P", "Q", "T", np.nan])].copy()

In [ ]:
df_prep["Term"].value_counts(dropna=False)

In [ ]:
df_prep = df_prep[df_prep["Term"].isin(["live", "skins", "bodies", "trophies", "specimens"])].copy()

## D.

In [ ]:
df_prep[["Importer", "Exporter"]].isna().values.any()

In [ ]:
df_prep.sample(10)

In [ ]:
df_prep [(df_prep["Importer reported quantity"] == np.nan) & (df_prep["Exporter reported quantity"] == np.nan)]

In [ ]:
df_prep["quantity"] = df_prep.apply(lambda x: np.nanmax(x[["Importer reported quantity", "Exporter reported quantity"]]), axis="columns")

## E.

In [ ]:
edgelist_raw = df_prep[["Importer", "Exporter", "quantity"]].copy()

In [ ]:
edgelist_unique = edgelist_raw.groupby(["Exporter", "Importer"], as_index=False).sum().sort_values("quantity", ascending=False).rename(columns={"quantity": "weight"})

In [ ]:
edgelist_unique.weight.sum()

## F.

In [ ]:
G = nx.from_pandas_edgelist(df=edgelist_unique, source="Exporter", target="Importer", edge_attr="weight", create_using=nx.MultiDiGraph())

In [ ]:
nx.draw_networkx(G, pos=nx.spring_layout(G, k=20))

## G.

In [ ]:
list_drawing_values = []
for i in G.edges(data=True):
    list_drawing_values.append(i[2]["weight"])

scaled_drawing_values = np.digitize(list_drawing_values, bins=np.arange(0,320000,40000))

cmap = plt.colormaps["RdYlGn_r"]
norm_class = plt.Normalize(vmin=min(scaled_drawing_values), vmax=max(scaled_drawing_values))
custom_edge_colors = cmap(norm_class(scaled_drawing_values))

In [ ]:
plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k=50, seed=0)
nx.draw_networkx_nodes(G, pos, node_color="black", node_size=1000)
nx.draw_networkx_labels(G, pos, font_color="white", font_size=18)
nx.draw_networkx_edges(G, pos, width=scaled_drawing_values, edge_color=custom_edge_colors, arrows=True, arrowsize=12, min_target_margin=20)
plt.axis("off")
plt.tight_layout()
plt.show()

## H.

In [ ]:
G.number_of_nodes()

In [ ]:
sorted(nx.centrality.degree_centrality(G).items(), key=lambda item: item[1], reverse=True)[:10]

---

Copyright © 2023 Rho Zeta AI Ltd. All rights reserved.